In [ ]:
# Panel B
import json
import matplotlib.pyplot as plt
rpneg = json.load(open("/Users/mitchjo/Analyses/HZV029_plasma_RP_neg/experiment.json"))
hilicpos = json.load(open("/Users/mitchjo/Analyses/HZV029_plasma_HILIC_pos/experiment.json"))

print("RPneg All Acquisitions:", len(rpneg["acquisitions"]))
print("HILICpos All Acquisitions:", len(hilicpos["acquisitions"]))
print("RPneg MS1 Acquisitions:", len([x for x in rpneg["acquisitions"] if x["_Acquisition__has_ms2"] is not True]))
print("HILICpos MS1 Acquisitions:", len([x for x in hilicpos["acquisitions"] if x["_Acquisition__has_ms2"] is not True]))



for title, data in zip(["HZV029 RP-", "HZV029 HILIC+"], [rpneg, hilicpos]):
    cmds = data["command_history"]
    total_time_elapsed = {}
    for i, cmd_1 in enumerate(cmds[:-1]):
        cmd_2 = cmds[i+1]
        time_1, cmd_1 = cmd_1.split(":")
        time_2, cmd_2 = cmd_2.split(":")
        cmd_1_str = cmd_1.split(";")
        if len(cmd_1_str) > 1:
            cmd_1_str = cmd_1.split(";")[1]
        else:
            cmd_1_str = cmd_1.split(";")[0]
        subcommand = cmd_2.split(";")[1]
        time_used = float(time_2) - float(time_1)

        if subcommand not in total_time_elapsed:
            if subcommand == 'finish':
                subcommand = 'report' # this is a bug fix for how the reporting was done.
            total_time_elapsed[subcommand] = 0
        total_time_elapsed[subcommand] += time_used / 60
    colors = {
        'assemble': 'lightgray',
        'asari': 'lightblue',
        'convert': 'lightblue',
        'blank_masking': 'lightcoral',
        'drop_samples': 'lightcoral',
        'drop_outliers': 'lightcoral',
        'drop_missing_features': 'lightcoral',
        'normalize': 'lightcoral',
        'batch_correct': 'lightcoral', 
        'log_transform': 'lightcoral',
        'interpolate': 'lightcoral',
        'impute': 'lightcoral',
        'build_empCpds': 'lightgreen',
        'l4_annotate': 'lightgreen',
        'l2_annotate': 'lightgreen',
        'L1_annotate_w_MS2': 'lightgreen',
        'map_ms2': 'lightgreen',
        'l1a_annotate': 'lightgreen',
        'l1b_annotate': 'lightgreen',
        'report': 'yellow',
        'finish': 'yellow' # any additional time assign to reporting
    }


    subtotal = 0
    for sub, dt in total_time_elapsed.items():
        if sub:
            plt.bar(title, dt, bottom=subtotal, color=colors[sub])
            subtotal += dt
    with open(title + "timings.csv", 'w+') as out_fh:
        print("Time Needed per Subcommand, " + title, file=out_fh)
        for sub, dt in total_time_elapsed.items():
            print(str(sub) + "," + str(dt), file=out_fh)
plt.show()
